In [7]:
from boto3 import session
from botocore.client import Config
from decouple import config
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime
from collections import ChainMap
import re

In [8]:
load_dotenv()

True

In [38]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [9]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [10]:
users = pd.read_csv('csv/users.csv')
products = pd.read_csv('csv/products.csv', delimiter=';')
localities = pd.read_csv('out/localities.csv')

In [11]:
usersDf = users[['Profile picture', 'Photo name', 'Username']]

In [12]:
def new_photo_name(value):
    return str(value[1])[-4:] + str(value[2])

In [13]:
users['new_name'] = usersDf.apply(new_photo_name,  axis = 1)

In [14]:
API_ACCESS_ID = config('ACCESS_ID')
API_SECRET_KEY = config('SECRET_KEY')
AWS_ENDPOINT = config('AWS_ENDPOINT')
AWS_CDN = config('AWS_CDN')
FULL_URL = config('FULL_URL')

In [15]:
session = session.Session()

In [16]:
client = session.client('s3',
                        endpoint_url=AWS_ENDPOINT,
                        aws_access_key_id=API_ACCESS_ID,
                        aws_secret_access_key=API_SECRET_KEY)

In [17]:
def rename_image(name, ext = 'webp'):
    if 'region' not in name or 'product' not in name:
        for index, data in users.iterrows():
            if users['Photo name'].iloc[index] == str(name):
                return [users['new_name'].iloc[index], ext]
        
    return [name, ext]

In [81]:
for index, image in enumerate(os.listdir("users")):
    name, ext = os.path.splitext(image)
    new_image = rename_image(name, ext)
    for i, j in enumerate(usersDf['Photo name']):
        if str(name) in str(j).lower():
            if usersDf['Profile picture'].iloc[i] == 'ANO':
                if new_image is not None:
                    client.upload_file(
                        'users/'+image, AWS_CDN, new_image[0]+new_image[1],
                        ExtraArgs={'ACL': 'public-read', 'ContentType': 'image/webp'}
                    )

In [82]:
for index, image in enumerate(os.listdir("images")):
    name, ext = os.path.splitext(image) 
    new_image = rename_image(name, ext)
    if new_image is not None:
            client.upload_file(
                'images/'+image, AWS_CDN, new_image[0]+new_image[1],
                ExtraArgs={'ACL': 'public-read', 'ContentType': 'image/webp'}
            )

In [18]:
columns = ['id', 'createdDate', 'updatedDate', 'url', 'localityId', 'productId', 'storyId', \
          'mime', 'order', 'creatorId', 'statusId', 'blogPostSectionId']

In [19]:
df = pd.DataFrame(columns = columns)

In [20]:
empty = []

In [21]:
def get(path):
    for index, file in enumerate(os.listdir(path)):
        name, ext = os.path.splitext(file)
        if path == 'users':
            for i, j in enumerate(usersDf['Photo name']):
                if str(name) in str(j).lower():
                    if usersDf['Profile picture'].iloc[i] == 'ANO':
                        empty.append(name)
        else:
            empty.append(name)

In [22]:
get("users")
get("images")

In [23]:
df = pd.DataFrame(data = empty)

In [24]:
df = pd.concat([df, pd.DataFrame(columns = columns)])

In [25]:
df.rename(columns = {0:'name'}, inplace = True)

In [26]:
def set_url(value):
    return FULL_URL + value + '.webp'

In [27]:
df['id'] = [i for i in range(1, df.shape[0]+1)]
df['createdDate'] = dt_string
df['updatedDate'] = dt_string
df['localityId'] = ''
df['productId'] = ''
df['storyId'] = ''
df['order'] = ''
df['creatorId'] = ''
df['statusId'] = 4
df['blogPostSectionId'] = ''
df['mime'] = 'image/webp'
df['new_name'] = df['name'].apply(rename_image)
df['new_name'] = df['new_name'].str[0]
df['url'] = df['new_name'].apply(set_url)

In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [29]:
df = df[:][df['name'] != '.DS_Store']

In [30]:
def get_product_id(value):
    if "product" in str(value):
        try:
            return int(re.search(r"\d", value).group())
        except:
            pass
    else:
        return ''
        
df['productId'] = df['name'].apply(get_product_id)

In [31]:
def get_locality_id(value):
    if "region" in str(value):
        for index, data in localities.iterrows():
            if localities['photoName'].iloc[index].lower().replace(' ', '_') == str(value).replace('region_', ''):
                return localities['id'].iloc[index]
        
df['localityId'] = df['name'].apply(get_locality_id)

In [32]:
df.to_csv('out/image.csv', index=False, encoding='utf-8')
users.to_csv('csv/users.csv', index=False, encoding='utf-8')

In [33]:
df.shape

(4583, 14)

In [42]:
df[['name', 'url', 'new_name']][df['name'] == '1808_698607097']

,name,url,new_name
417,1808_698607097,https://trynego-cdn.ams3.digitaloceanspaces.com/7097YvesStef.webp,7097YvesStef
